#### DESCRIPTION
    Azure Subscription Cleanup Script

    The script is designed for Subscription cleanup and maintenance based on the configuration file.

    Configuration File:
        The configuration file is a YAML file that will manage the settings for the script.
        The configuration file will manage the following settings:
            - Subscription Information
            - Permissions
            - Maintenance Window

	

#### NOTES

**[Original Author]**
* Michael Arroyo

**[Original Build Version]**
* 1.0.0.20241210 (Major.Minor.Patch.Date<YYYYMMDD>)

**[Latest Author]**
* Michael Arroyo

**[Latest Build Version]**
* 1.2.0.20241219 (Major.Minor.Patch.Date<YYYYMMDD>)

**[Comments]**
* .

**[Python Compatibility / Tested On]**
* Python 3.10.15

**[Forked Project]**
* .

**[Dependencies]**
* **pyyaml** # PyYAML is a YAML parser and emitter for Python
* **rich** # Rich is a Python library for rich text and beautiful formatting in the terminal
* **az** # The az module is a wrapper for the Azure CLI
* **os** # The os module provides a way to use operating system dependent functionality
* **classes.class_cloudobj** # Import the CloudObj class from the class_cloudobj.py file
* **classes.class_maintenance** # Import the MaintenanceObj class from the class_maintenance
* **classes.class_removalobj** # Import the RemovalObj class from the class_removalobj.py file
* **timestamp** # Import the timestamp function from the timestamp.py file
* **argparse** # The argparse module makes it easy to write user-friendly command-line interfaces
* **isNotebook** # Import the is_notebook detection method from the isNotebook.py file

#### Build Notes

* 1.0.0.20241210
    * [Michael Arroyo] Initial Build
* 1.1.0.20241216
    * [Michael Arroyo] Port main.py to Jupyter Notebook
    * [Michael Arroyo] Update main.py as the Jupyter Notebook caller while passing the necessary parameters
* 1.2.0.20241219
    * [Michael Arroyo] Add support for Jupyter Notebook detection
    * [Michael Arroyo] Add Mockup variables to use while only in the Jupyter Notebook
    * [Michael Arroyo] Save output to notebook

#### Examples

* *EXAMPLE*
    * **Command**: python main.py --config ./config/Ares-Sec-Blue.yaml
    * **Description**: Run the cleanup script using the Ares-Sec-Blue configuration file
    * **Notes**: The process uses ipython as the core interpreter for the Jupyter notebook.
    * **Output**: N/A
<br></br>
* *EXAMPLE*
    * **Command**: jupyter nbconvert --to python --execute main.ipynb --output main_notebook.py && python main_notebook.py --config ./config/Ares-Sec-Blue.yaml && rm main_notebook.py
    * **Description**: Run the cleanup script using the Ares-Sec-Blue configuration file
    * **Notes**: Convert the Jupyter notebook to a Python file, Using Python run the Python script with the necessary arguments.  Remove the file after a successful return.
    * **Output**:

***Import Core Modules***

In [1]:
from pathlib import Path
import os
import argparse

***Import Third-Party Modules***

In [2]:
import yaml
from rich import print
from az.cli import az
from time import gmtime, strftime

***Import Custom Modules and Classes***

In [6]:
from classes.class_cloudobj import CloudObj
from classes.class_maintenanceobj import MaintenanceObj
from classes.class_removalobj import RemovalObj
from bin.timestamp import timestamp
from bin.isNotebook import is_notebook

***Set Script Variables***

**Note**:  If you are running this session as a Mockup to test the notebook.  You will have to declare that s_config and s_bypass variables.  These variables are queried from the session environment variables or updated via the main.py at runtime.  This section will only run if your running the code in an interactive Jupyter Notebook session.

In [13]:
if is_notebook():
    timestamp("Tracking as notebook", "warning")
    os.environ['s_config'] =  'config/Ares-Sec-Red.yaml'
    os.environ['s_bypass'] = str(False)

[!] 2024-12-19 12:53:58 -0500: Tracking as notebook


In [14]:
logfile = f'logs/{strftime("%Y.%m.%d_%H.%M.%S_%z")}.log'
timestamp("Loading Script Variables", "alert", logfile)
session_config = os.getenv('s_config')
if session_config is None:
    timestamp("Failed to load the config variable (s_config)\nMake sure to set an OS environment variable for s_config to the path of the config file when running this directly from the Notebook", "error", logfile)
try:
    session_bypass = eval(os.getenv('s_bypass'))
except:
    session_bypass = None
    timestamp("Failed to load the bypass variable (s_bypass)\nMake sure to set an OS environment variable for s_bypass to either True or False when running this directly from the Notebook", "error", logfile)
remove_results = []
timestamp("Loading Script Variables - Done", "alert", logfile)

if session_config is None and session_bypass is None:
    raise Exception("Failed to load the config file and bypass variable")

timestamp(f"logfile: {logfile}", "alert", logfile)
timestamp(f"session_config: {session_config}", "alert", logfile)
timestamp(f"session_bypass: {session_bypass}", "alert", logfile)

[+] 2024-12-19 12:54:06 -0500: Loading Script Variables
[+] 2024-12-19 12:54:06 -0500: Loading Script Variables - Done
[+] 2024-12-19 12:54:06 -0500: logfile: logs/2024.12.19_12.54.06_-0500.log
[+] 2024-12-19 12:54:06 -0500: session_config: config/Ares-Sec-Red.yaml
[+] 2024-12-19 12:54:06 -0500: session_bypass: True


***Import Yaml Config File***

In [9]:
timestamp("Loading Config File", "alert", logfile)
with open(session_config, 'r') as file:
    config = yaml.safe_load(file)
timestamp("Loading Config File - Done", "alert", logfile)

[+] 2024-12-19 12:52:07 -0500: Loading Config File
[+] 2024-12-19 12:52:07 -0500: Loading Config File - Done


***Query Config File Meta***

In [10]:
timestamp("Process Config File Meta", "alert", logfile)
az(f'account set --subscription {config["settings"]["subscription"]["name"]}')

connection = CloudObj(config["settings"]["subscription"]["name"],
            config["settings"]["subscription"]["id"],
            config["settings"]["subscription"]["tenantId"],
            config["settings"]["permissions"]["hasAccess"])

maintenance = MaintenanceObj(config["settings"]["maintenance"]["start_day"],
            config["settings"]["maintenance"]["end_day"],
            config["settings"]["maintenance"]["start_time"],
            config["settings"]["maintenance"]["end_time"])
timestamp("Process Config File Meta - Done", "alert", logfile)

[+] 2024-12-19 12:52:12 -0500: Process Config File Meta
[+] 2024-12-19 12:52:15 -0500: Process Config File Meta - Done


***Filter Bypass Process***

In [16]:
if session_bypass == True:
    timestamp("Bypass was enabled", "alert", logfile)
    timestamp("Setting Bypass arguments", "alert", logfile)
    maintenance.start_day = 1
    maintenance.end_day = 31
    maintenance.start_time = "00:00"
    maintenance.end_time = "23:59"
    bypass_id = az("account show")[1]['id']
    connection.hasAccess.append(az("account show")[1]['id'])
    timestamp("Setting Bypass arguments - Done", "alert", logfile)

[+] 2024-12-19 12:55:21 -0500: Bypass was enabled
[+] 2024-12-19 12:55:21 -0500: Setting Bypass arguments
[+] 2024-12-19 12:55:21 -0500: Setting Bypass arguments - Done


***Main***

* **Step (1)** - Validate Connection to Azure
* **Step (2)** - Validate the current user connected to Azure is also permissioned via the Config file to run the clean process
* **Step (3)** - Validate the Maintenance windows.  Make sure the current runtime of the script falls into the Maintenance window.

In [ ]:
timestamp("Processing Azure Subscription Cleanup", "alert", logfile)
timestamp("Validating Connection and Maintenance Window", "alert", logfile)
if connection.validate() and connection.validateUser() and maintenance.validate():
    timestamp("You have access to the subscription and the maintenance window is open.", "alert", logfile)

    # Group Removal
    timestamp("Querying Azure Tenant for Resource Groups", "alert", logfile)
    group_scan_exitcode, group_scan_results, group_scan_logs = az("group list")
    timestamp("Querying Azure Tenant for Resource Groups - Done", "alert", logfile)

    timestamp("Processing Cleanup", "alert", logfile)
    for group in group_scan_results:
        current_group_obj = RemovalObj(group["name"],
            group["type"],
            group["name"])

        current_group_removal = az(f'group delete --name "{current_group_obj.name}" --yes')

        if current_group_removal.exit_code == 0:
            current_group_obj.removed = True
        else:
            current_group_obj.log = current_group_removal.log

        current_group_obj.exitcode = current_group_removal.exit_code

        remove_results.append(current_group_obj)
    timestamp("Processing Cleanup - Done", "alert", logfile)
    if group_scan_results == []:
        timestamp("No Resource Groups found to remove", "alert", logfile)
else:
    timestamp("You do not have access to the subscription or the maintenance window is closed.", "error", logfile)
timestamp("Processing Azure Subscription Cleanup - Done", "alert", logfile)

[+] 2024-12-19 12:55:30 -0500: Processing Azure Subscription Cleanup
[+] 2024-12-19 12:55:30 -0500: Validating Connection and Maintenance Window
[+] 2024-12-19 12:55:30 -0500: You have access to the subscription and the maintenance window is open.
[+] 2024-12-19 12:55:30 -0500: Querying Azure Tenant for Resource Groups
[+] 2024-12-19 12:55:33 -0500: Querying Azure Tenant for Resource Groups - Done
[+] 2024-12-19 12:55:33 -0500: Processing Cleanup
